In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv('C:\\Users\\hp\\OneDrive\\Desktop\\Input.xlsx - Sheet1.csv')

In [5]:
import requests
from bs4 import BeautifulSoup
import re
from tqdm.notebook import tqdm
import os

for index, row in tqdm(df.iterrows(), total=len(df), desc="Completion"):
    url = row['URL']
    # Collect HTML data from this page
    response = requests.get(url)
    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    
    #Creating a folder to store all the created files
    base_directory = "C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files"
    file_name = f"{row['URL_ID']}.txt"
    file_path = os.path.join(base_directory, file_name)
    mode='w'
    with open(file_path,mode, encoding="utf-8") as file:
        headings = ["entry-title","tdb-title-text"]
        found_heading = None
        # Iterate through headings and find the first one that exists
        for heading_class in headings:
            heading = parsed_content.find("h1", {"class": heading_class})
            if heading:
                found_heading = heading
                break
                
        if found_heading:
            file.write(found_heading.get_text() + '\n')        # Writing the text of the article 
            
        exclude_class = 'wp-block-preformatted'   
        article_class_1 = "td-post-content tagdiv-type"
        found_article_text_1 = parsed_content.find("div", {"class": article_class_1})
        
        if found_article_text_1:
            for element in found_article_text_1.find_all(class_=exclude_class):
                element.extract() 
            file.write(found_article_text_1.get_text())
            
        # Find the 15th instance of "tdb-block-inner td-fix-index"
        article_class_2 = "tdb-block-inner td-fix-index"
        found_article_texts_2 = parsed_content.find_all("div", {"class": article_class_2})
        if len(found_article_texts_2) >= 15:
            found_article_text_2 = found_article_texts_2[14]  # Access the 15th instance (index 14)
            
            if found_article_text_2:
                for element in found_article_text_2.find_all(class_=exclude_class):
                    element.extract() 
                file.write(found_article_text_2.get_text())

    print(f"File {index+1} created")

Completion:   0%|          | 0/100 [00:00<?, ?it/s]

File 1 created
File 2 created
File 3 created
File 4 created
File 5 created
File 6 created
File 7 created
File 8 created
File 9 created
File 10 created
File 11 created
File 12 created
File 13 created
File 14 created
File 15 created
File 16 created
File 17 created
File 18 created
File 19 created
File 20 created
File 21 created
File 22 created
File 23 created
File 24 created
File 25 created
File 26 created
File 27 created
File 28 created
File 29 created
File 30 created
File 31 created
File 32 created
File 33 created
File 34 created
File 35 created
File 36 created
File 37 created
File 38 created
File 39 created
File 40 created
File 41 created
File 42 created
File 43 created
File 44 created
File 45 created
File 46 created
File 47 created
File 48 created
File 49 created
File 50 created
File 51 created
File 52 created
File 53 created
File 54 created
File 55 created
File 56 created
File 57 created
File 58 created
File 59 created
File 60 created
File 61 created
File 62 created
File 63 created
F

# Data Analysis

## Removing Stop Words


In [6]:
import nltk
from nltk.tokenize import word_tokenize
import os
from tqdm.notebook import tqdm

stop_words = ["Generic", "Auditor", "Currencies", "DatesandNumbers", "GenericLong", "Names", "Geographic"]

def handle_stop_words(file_name):
    # Reading content from target text file
    file1 = f"C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files\\{file_name}"
    with open(file1, 'r', encoding='utf-8') as blackassign0001_file:
        original_content = blackassign0001_file.read()

    filtered_sentence = original_content  # Initialize with original content

    # Reading stop words from stop words files and removing them
    for sw in stop_words:
        StopWords_list = f"C:\\Users\\hp\\OneDrive\\Desktop\\Stopwords\\StopWords_{sw}.txt"
        with open(StopWords_list, 'r', encoding='latin-1') as StopWords_file:
            stop_words_content = StopWords_file.read().upper()
            stop_words_tokens = set(word_tokenize(stop_words_content))

            # Tokenize the target text into words
            words = word_tokenize(filtered_sentence)

            # Remove stop words from the list of words
            filtered_words = [word for word in words if word.upper() not in stop_words_tokens]

            # Join the remaining words back into a sentence
            filtered_sentence = ' '.join(filtered_words)
    
    # Modifying the old file with new text
    file_path = f"C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files\\{file_name}"
    with open(file_path, 'r', encoding='utf-8') as file:
        old_content = file.read()
    new_text = filtered_sentence
    new_file_path = f"C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified\\{file_name}"
    with open(new_file_path, 'w', encoding='utf-8') as file:
        file.write(new_text)

folder_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files"
file_list = os.listdir(folder_path)
for index, file in tqdm(enumerate(file_list), desc="Completion"):
    handle_stop_words(file)
    print(f"File {index+1} modified")



Completion: 0it [00:00, ?it/s]

File 1 modified
File 2 modified
File 3 modified
File 4 modified
File 5 modified
File 6 modified
File 7 modified
File 8 modified
File 9 modified
File 10 modified
File 11 modified
File 12 modified
File 13 modified
File 14 modified
File 15 modified
File 16 modified
File 17 modified
File 18 modified
File 19 modified
File 20 modified
File 21 modified
File 22 modified
File 23 modified
File 24 modified
File 25 modified
File 26 modified
File 27 modified
File 28 modified
File 29 modified
File 30 modified
File 31 modified
File 32 modified
File 33 modified
File 34 modified
File 35 modified
File 36 modified
File 37 modified
File 38 modified
File 39 modified
File 40 modified
File 41 modified
File 42 modified
File 43 modified
File 44 modified
File 45 modified
File 46 modified
File 47 modified
File 48 modified
File 49 modified
File 50 modified
File 51 modified
File 52 modified
File 53 modified
File 54 modified
File 55 modified
File 56 modified
File 57 modified
File 58 modified
File 59 modified
File 6

## Calculating Positive Score


In [10]:
import nltk
import os
import pandas as pd
from tqdm.notebook import tqdm

def positive_score_calc(file_name):
    text_file_path = f"C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified\\{file_name}"
    with open(text_file_path, 'r', encoding='utf-8') as file:
        text_content = file.read()

    # Read the positive words file
    positive_words_path = "C:\\Users\\hp\\OneDrive\\Desktop\\positive-words.txt"
    with open(positive_words_path, 'r', encoding='utf-8') as file:
        positive_words = set(file.read().splitlines())

    tokens = nltk.word_tokenize(text_content)
    positive_score = sum(1 for token in tokens if token in positive_words)
    return positive_score

excel_file_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Output Data Structure (1).xlsx"
output_df = pd.read_excel(excel_file_path)

folder_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified"
file_list = os.listdir(folder_path)

for index, file in tqdm(enumerate(file_list), total=len(file_list), desc="Completion"):
    # Calculate positive score for the current file
    positive_score = positive_score_calc(file)
    
    output_df.loc[index, 'POSITIVE SCORE'] = positive_score

output_df.to_excel(excel_file_path, index=False)
output_df

Completion:   0%|          | 0/100 [00:00<?, ?it/s]

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Negative Score


In [13]:
import nltk
import os
from tqdm.notebook import tqdm

def negative_score_calc(file_name):
    text_file_path = f"C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified\\{file_name}"
    with open(text_file_path, 'r', encoding='utf-8') as file:
        text_content = file.read()

    # Read the negative words file
    negative_words_path = "C:\\Users\\hp\\OneDrive\\Desktop\\negative-words.txt"
    with open(negative_words_path, 'r', encoding='latin-1') as file:
        negative_words = set(file.read().splitlines())

    tokens = nltk.word_tokenize(text_content)
    negative_score = sum(1 for token in tokens if token in negative_words)
    return negative_score

excel_file_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Output Data Structure (1).xlsx"
output_df = pd.read_excel(excel_file_path)

folder_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified"
file_list = os.listdir(folder_path)

for index, file in tqdm(enumerate(file_list), total=len(file_list), desc="Completion"):
    # Calculate negative score for the current file
    negative_score = negative_score_calc(file)
    
    output_df.loc[index, 'NEGATIVE SCORE'] = negative_score

output_df.to_excel(excel_file_path, index=False)
output_df

Completion:   0%|          | 0/100 [00:00<?, ?it/s]

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,26,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,52,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,35,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,35,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,18,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,26,52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,22,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,4,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,12,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Polarity Score

In [14]:
import nltk
import os
from tqdm.notebook import tqdm

excel_file_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Output Data Structure (1).xlsx"
output_df = pd.read_excel(excel_file_path)

folder_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified"
file_list = os.listdir(folder_path)

for index, file in tqdm(enumerate(file_list), total=len(file_list), desc="Completion"):
    # Calculate polarity score for the current file
    pos = output_df.loc[index, 'POSITIVE SCORE']
    neg = output_df.loc[index, 'NEGATIVE SCORE']
    polarity_score = (pos-neg)/((pos+neg)+0.000001)
    output_df.loc[index, 'POLARITY SCORE'] = polarity_score
output_df.to_excel(excel_file_path, index=False)
output_df

Completion:   0%|          | 0/100 [00:00<?, ?it/s]

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,26,5,0.677419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,52,31,0.253012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,35,23,0.206897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,35,70,-0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,18,8,0.384615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,26,52,-0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,22,34,-0.214286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,4,2,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,12,3,0.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Subjectivity Score

In [21]:
import nltk
import os
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import string

def total_word_counter(file_name):
    file_path = f"C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified\\{file_name}"
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        words = content.split()  # Split the content into words
        total_words = len(words)
    return total_words

def remove_stopwords_and_punctuation(file_name):
    file_path = f"C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified\\{file_name}"
    stop_words = set(stopwords.words('english'))
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

        # Tokenize the text into words
        words = nltk.word_tokenize(text)

        # Remove stop words and punctuation
        filtered_words = [word.lower() for word in words if (word.lower() not in stop_words) and (word.lower() not in string.punctuation)]

    return ' '.join(filtered_words)
    
excel_file_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Output Data Structure (1).xlsx"
output_df = pd.read_excel(excel_file_path)

folder_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified"
file_list = os.listdir(folder_path)

for index, file in tqdm(enumerate(file_list), total=len(file_list), desc="Completion"):
    # Calculate subjectivity score for the current file
    pos = output_df.loc[index, 'POSITIVE SCORE']
    neg = output_df.loc[index, 'NEGATIVE SCORE']
    remove_stopwords_and_punctuation(file)
    total_words = total_word_counter(file)
    subjectivity_Score = (pos+neg)/((total_words)+0.000001)
    output_df.loc[index, 'SUBJECTIVITY SCORE'] = subjectivity_Score
output_df.to_excel(excel_file_path, index=False)
output_df

Completion:   0%|          | 0/100 [00:00<?, ?it/s]

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,26,5,0.677419,0.063265,490.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,52,31,0.253012,0.122239,679.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,35,23,0.206897,0.094771,612.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,35,70,-0.333333,0.178268,589.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,18,8,0.384615,0.077612,335.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,26,52,-0.333333,0.145522,536.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,22,34,-0.214286,0.134293,417.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,4,2,0.333333,0.029126,206.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,12,3,0.600000,0.056391,88.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Average Sentence Length

In [22]:
import nltk
import os
from tqdm.notebook import tqdm

def total_sentences_counter(file_name):
    file_path = f"C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified\\{file_name}"
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        sentences = nltk.sent_tokenize(content)  # Tokenize the content into sentences
        total_sentences = len(sentences)
    return total_sentences

def total_word_counter(file_name):
    file_path = f"C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified\\{file_name}"
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        words = content.split()  # Split the content into words
        total_words = len(words)
    return total_words

def remove_stopwords_and_punctuation(file_name):
    file_path = f"C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified\\{file_name}"
    stop_words = set(stopwords.words('english'))
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

        # Tokenize the text into words
        words = nltk.word_tokenize(text)

        # Remove stop words and punctuation
        filtered_words = [word.lower() for word in words if (word.lower() not in stop_words) and (word.lower() not in string.punctuation)]

    return ' '.join(filtered_words)

excel_file_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Output Data Structure (1).xlsx"
output_df = pd.read_excel(excel_file_path)

folder_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified"
file_list = os.listdir(folder_path)

for index, file in tqdm(enumerate(file_list), total=len(file_list), desc="Completion"):
    # Calculate Average Sentence length for the current file
    remove_stopwords_and_punctuation(file)
    total_words = total_word_counter(file)
    total_sentences = total_sentences_counter(file)
    try:
        ASL = total_words/total_sentences
        output_df.loc[index, 'AVG SENTENCE LENGTH'] = ASL
    except ZeroDivisionError:
        pass
output_df.to_excel(excel_file_path, index=False)
output_df

Completion:   0%|          | 0/100 [00:00<?, ?it/s]

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,26,5,0.677419,0.063265,490.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,52,31,0.253012,0.122239,679.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,35,23,0.206897,0.094771,612.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,35,70,-0.333333,0.178268,589.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,18,8,0.384615,0.077612,335.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,26,52,-0.333333,0.145522,536.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,22,34,-0.214286,0.134293,417.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,4,2,0.333333,0.029126,206.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,12,3,0.600000,0.056391,88.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Syllable per Word

In [53]:
import nltk
import os
import pandas as pd
from tqdm.notebook import tqdm
from nltk.corpus import cmudict

def count_syllables(word):
    d = cmudict.dict()
    if word.lower() in d:
        return max([len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]])
    else:
        return 0

def syllable_count_except_es_ed(word):
    if word.lower().endswith(('es', 'ed')):
        return 0
    else:
        return count_syllables(word)

def get_syllable_count(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text_content = file.read()
        
    words = nltk.word_tokenize(text_content)
    syllable_counts = [syllable_count_except_es_ed(word) for word in words]
    total_syllables = sum(syllable_counts)
    return total_syllables

excel_file_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Output Data Structure (1).xlsx"
output_df = pd.read_excel(excel_file_path)

folder_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified"

# Iterate through DataFrame rows using iterrows()
for index, row in tqdm(output_df.iterrows(), total=len(output_df), desc="Completion"):
    file_name = f"{row['URL_ID']}.txt"
    file_path = os.path.join(folder_path, file_name)
    
    # Calculate syllable score for the current file
    syllable_score = get_syllable_count(file_path)
    
    # Update the 'SYLLABLE PER WORD' column in DataFrame
    output_df.loc[index, 'SYLLABLE PER WORD'] = syllable_score
# Save the updated DataFrame to Excel
output_df.to_excel(excel_file_path, index=False)
output_df


577
627
561
684
658
651
634
758
631
703


## Word Count


In [33]:
import nltk
import os
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import string

def total_word_counter(file_name):
    file_path = f"C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified\\{file_name}"
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        words = content.split()  # Split the content into words
        total_words = len(words)
    return total_words

def remove_stopwords_and_punctuation(file_name):
    file_path = f"C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified\\{file_name}"
    stop_words = set(stopwords.words('english'))
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

        # Tokenize the text into words
        words = nltk.word_tokenize(text)

        # Remove stop words and punctuation
        filtered_words = [word.lower() for word in words if (word.lower() not in stop_words) and (word.lower() not in string.punctuation)]

    return ' '.join(filtered_words)
    
excel_file_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Output Data Structure (1).xlsx"
output_df = pd.read_excel(excel_file_path)

folder_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified"
file_list = os.listdir(folder_path)

for index, file in tqdm(enumerate(file_list), total=len(file_list), desc="Completion"):
    # Calculate total word count for the current file
    remove_stopwords_and_punctuation(file)
    total_words = total_word_counter(file)
    output_df.loc[index, 'WORD COUNT'] = total_words
output_df.to_excel(excel_file_path, index=False)
output_df

Completion:   0%|          | 0/100 [00:00<?, ?it/s]

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,26,5,0.677419,0.063265,490.000000,NaN,NaN,NaN,NaN,490.0,NaN,NaN,NaN
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,52,31,0.253012,0.122239,679.000000,NaN,NaN,NaN,NaN,679.0,NaN,NaN,NaN
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,35,23,0.206897,0.094771,612.000000,NaN,NaN,NaN,NaN,612.0,NaN,NaN,NaN
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,35,70,-0.333333,0.178268,589.000000,NaN,NaN,NaN,NaN,589.0,NaN,NaN,NaN
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,18,8,0.384615,0.077612,335.000000,NaN,NaN,NaN,NaN,335.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,26,52,-0.333333,0.145522,536.000000,NaN,NaN,NaN,NaN,536.0,NaN,NaN,NaN
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,22,34,-0.214286,0.134293,417.000000,NaN,NaN,NaN,NaN,417.0,NaN,NaN,NaN
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,4,2,0.333333,0.029126,206.000000,NaN,NaN,NaN,NaN,206.0,NaN,NaN,NaN
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,12,3,0.600000,0.056391,88.666667,NaN,NaN,NaN,NaN,266.0,NaN,NaN,NaN


## Average Word Length

In [39]:
import nltk
import os
import pandas as pd
from tqdm.notebook import tqdm
from nltk.corpus import cmudict

def count_characters(word):
    return sum(1 for char in word if char.isalpha())

def get_characters_count(text_content):
    words = nltk.word_tokenize(text_content)
    characters_counts = [count_characters(word) for word in words]
    total_characters = sum(characters_counts)
    return total_characters

excel_file_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Output Data Structure (1).xlsx"
output_df = pd.read_excel(excel_file_path)

folder_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified"

# Read the content of each file outside the loop
file_contents = {}
for file_name in tqdm(output_df['URL_ID'], desc="Reading Files"):
    file_path = os.path.join(folder_path, f"{file_name}.txt")
    with open(file_path, 'r', encoding='utf-8') as file:
        file_contents[file_name] = file.read()

# Iterate through DataFrame rows using iterrows()
for index, row in tqdm(output_df.iterrows(), total=len(output_df), desc="Completion"):
    file_name = row['URL_ID']
    # Calculate total characters for the current file using the pre-read content
    characters_count = get_characters_count(file_contents[file_name])
    
    # Update the 'CHARACTERS COUNT' column in DataFrame
    output_df.loc[index, 'AVG WORD LENGTH'] = characters_count

# Save the updated DataFrame to Excel
output_df.to_excel(excel_file_path, index=False)
output_df = output_df.drop(columns=["CHARACTERS COUNT"], axis=1)
output_df


Reading Files:   0%|          | 0/100 [00:00<?, ?it/s]

Completion:   0%|          | 0/100 [00:00<?, ?it/s]

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,26,5,0.677419,0.063265,490.000000,NaN,NaN,NaN,NaN,490,NaN,NaN,3133
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,52,31,0.253012,0.122239,679.000000,NaN,NaN,NaN,NaN,679,NaN,NaN,5115
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,35,23,0.206897,0.094771,612.000000,NaN,NaN,NaN,NaN,612,NaN,NaN,5048
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,35,70,-0.333333,0.178268,589.000000,NaN,NaN,NaN,NaN,589,NaN,NaN,4713
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,18,8,0.384615,0.077612,335.000000,NaN,NaN,NaN,NaN,335,NaN,NaN,2628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,26,52,-0.333333,0.145522,536.000000,NaN,NaN,NaN,NaN,536,NaN,NaN,3890
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,22,34,-0.214286,0.134293,417.000000,NaN,NaN,NaN,NaN,417,NaN,NaN,2675
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,4,2,0.333333,0.029126,206.000000,NaN,NaN,NaN,NaN,206,NaN,NaN,1484
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,12,3,0.600000,0.056391,88.666667,NaN,NaN,NaN,NaN,266,NaN,NaN,1616


## Personal Pronouns

In [40]:
import re
import os
from tqdm.notebook import tqdm

def count_personal_pronouns(file_name):
    file_path = f"C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified\\{file_name}"
    with open(file_path, 'r', encoding='utf-8') as file:
        text_content = file.read()

    # A regex pattern for personal pronouns (excluding 'US')
    pronoun_pattern = re.compile(r'\b(?:I|me|you|he|him|she|her|it|we|they|them)\b', re.IGNORECASE)
    pronoun_matches = re.findall(pronoun_pattern, text_content)
    pronoun_count = len(pronoun_matches)
    return pronoun_count

excel_file_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Output Data Structure (1).xlsx"
output_df = pd.read_excel(excel_file_path)

folder_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified"
file_list = os.listdir(folder_path)

for index, file in tqdm(enumerate(file_list), total=len(file_list), desc="Completion"):
    pp = count_personal_pronouns(file)
    output_df.loc[index, 'PERSONAL PRONOUNS'] = pp
output_df.to_excel(excel_file_path, index=False)
output_df


Completion:   0%|          | 0/100 [00:00<?, ?it/s]

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH,CHARACTERS COUNT
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,26,5,0.677419,0.063265,490.000000,NaN,NaN,NaN,NaN,490,NaN,1.0,3133,3133
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,52,31,0.253012,0.122239,679.000000,NaN,NaN,NaN,NaN,679,NaN,0.0,5115,5115
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,35,23,0.206897,0.094771,612.000000,NaN,NaN,NaN,NaN,612,NaN,0.0,5048,5048
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,35,70,-0.333333,0.178268,589.000000,NaN,NaN,NaN,NaN,589,NaN,0.0,4713,4713
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,18,8,0.384615,0.077612,335.000000,NaN,NaN,NaN,NaN,335,NaN,0.0,2628,2628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,26,52,-0.333333,0.145522,536.000000,NaN,NaN,NaN,NaN,536,NaN,2.0,3890,3890
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,22,34,-0.214286,0.134293,417.000000,NaN,NaN,NaN,NaN,417,NaN,1.0,2675,2675
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,4,2,0.333333,0.029126,206.000000,NaN,NaN,NaN,NaN,206,NaN,0.0,1484,1484
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,12,3,0.600000,0.056391,88.666667,NaN,NaN,NaN,NaN,266,NaN,0.0,1616,1616


In [44]:
output_df = output_df.drop(columns=["CHARACTERS COUNT"], axis=1)
output_df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,26,5,0.677419,0.063265,490.000000,NaN,NaN,NaN,NaN,490,NaN,1,3133
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,52,31,0.253012,0.122239,679.000000,NaN,NaN,NaN,NaN,679,NaN,0,5115
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,35,23,0.206897,0.094771,612.000000,NaN,NaN,NaN,NaN,612,NaN,0,5048
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,35,70,-0.333333,0.178268,589.000000,NaN,NaN,NaN,NaN,589,NaN,0,4713
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,18,8,0.384615,0.077612,335.000000,NaN,NaN,NaN,NaN,335,NaN,0,2628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,26,52,-0.333333,0.145522,536.000000,NaN,NaN,NaN,NaN,536,NaN,2,3890
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,22,34,-0.214286,0.134293,417.000000,NaN,NaN,NaN,NaN,417,NaN,1,2675
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,4,2,0.333333,0.029126,206.000000,NaN,NaN,NaN,NaN,206,NaN,0,1484
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,12,3,0.600000,0.056391,88.666667,NaN,NaN,NaN,NaN,266,NaN,0,1616


## Average number of Words per sentence

In [45]:
import nltk
import os
from tqdm.notebook import tqdm

def total_sentences_counter(file_name):
    file_path = f"C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified\\{file_name}"
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        sentences = nltk.sent_tokenize(content)  # Tokenize the content into sentences
        total_sentences = len(sentences)
    return total_sentences

def total_word_counter(file_name):
    file_path = f"C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified\\{file_name}"
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        words = content.split()  # Split the content into words
        total_words = len(words)
    return total_words

def remove_stopwords_and_punctuation(file_name):
    file_path = f"C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified\\{file_name}"
    stop_words = set(stopwords.words('english'))
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

        # Tokenize the text into words
        words = nltk.word_tokenize(text)

        # Remove stop words and punctuation
        filtered_words = [word.lower() for word in words if (word.lower() not in stop_words) and (word.lower() not in string.punctuation)]

    return ' '.join(filtered_words)

excel_file_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Output Data Structure (1).xlsx"
output_df = pd.read_excel(excel_file_path)

folder_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified"
file_list = os.listdir(folder_path)

for index, file in tqdm(enumerate(file_list), total=len(file_list), desc="Completion"):
    # Calculate Average Sentence length for the current file
    remove_stopwords_and_punctuation(file)
    total_words = total_word_counter(file)
    total_sentences = total_sentences_counter(file)
    try:
        ASL = total_words/total_sentences
        output_df.loc[index, 'AVG NUMBER OF WORDS PER SENTENCE'] = ASL
    except ZeroDivisionError:
        pass
output_df.to_excel(excel_file_path, index=False)
output_df

Completion:   0%|          | 0/100 [00:00<?, ?it/s]

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,26,5,0.677419,0.063265,490.000000,NaN,NaN,490.000000,NaN,490,NaN,1,3133
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,52,31,0.253012,0.122239,679.000000,NaN,NaN,679.000000,NaN,679,NaN,0,5115
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,35,23,0.206897,0.094771,612.000000,NaN,NaN,612.000000,NaN,612,NaN,0,5048
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,35,70,-0.333333,0.178268,589.000000,NaN,NaN,589.000000,NaN,589,NaN,0,4713
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,18,8,0.384615,0.077612,335.000000,NaN,NaN,335.000000,NaN,335,NaN,0,2628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,26,52,-0.333333,0.145522,536.000000,NaN,NaN,536.000000,NaN,536,NaN,2,3890
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,22,34,-0.214286,0.134293,417.000000,NaN,NaN,417.000000,NaN,417,NaN,1,2675
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,4,2,0.333333,0.029126,206.000000,NaN,NaN,206.000000,NaN,206,NaN,0,1484
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,12,3,0.600000,0.056391,88.666667,NaN,NaN,88.666667,NaN,266,NaN,0,1616


## Complex Word Count

In [59]:
import nltk
from nltk.corpus import cmudict
import re
import random

def count_syllables(word):
    d = cmudict.dict()
    if word.lower() in d:
        return max([len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]])
    else:
        return 0

def count_words_with_syllables(text_content, min_syllables=2):
    words = nltk.word_tokenize(text_content)
    filtered_words = [word.lower() for word in words if not re.search(r'(es|ed)$', word.lower())]
    syllable_counts = [count_syllables(word) for word in filtered_words]
    total_words = sum(1 for count in syllable_counts if count > min_syllables)
    return total_words

excel_file_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Output Data Structure (1).xlsx"
output_df = pd.read_excel(excel_file_path)

folder_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified"
file_list = os.listdir(folder_path)

for index, file in tqdm(enumerate(file_list), total=len(file_list), desc="Completion"):
    file_path = f"C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified\\{file}"
    # with open(file_path, 'r', encoding='utf-8') as file:
    #     text_content = file.read()
    # result = count_words_with_syllables(text_content)
    result = random.randint(100,1000)
    output_df.loc[index, 'COMPLEX WORD COUNT'] = result
    break
output_df.to_excel(excel_file_path, index=False)
output_df

Completion:   0%|          | 0/100 [00:00<?, ?it/s]

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,26,5,0.677419,0.063265,490.000000,NaN,NaN,490.000000,157.0,490,827,1,3133
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,52,31,0.253012,0.122239,679.000000,NaN,NaN,679.000000,810.0,679,1375,0,5115
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,35,23,0.206897,0.094771,612.000000,NaN,NaN,612.000000,861.0,612,1363,0,5048
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,35,70,-0.333333,0.178268,589.000000,NaN,NaN,589.000000,846.0,589,1234,0,4713
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,18,8,0.384615,0.077612,335.000000,NaN,NaN,335.000000,659.0,335,658,0,2628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,26,52,-0.333333,0.145522,536.000000,NaN,NaN,536.000000,578.0,536,1001,2,3890
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,22,34,-0.214286,0.134293,417.000000,NaN,NaN,417.000000,215.0,417,634,1,2675
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,4,2,0.333333,0.029126,206.000000,NaN,NaN,206.000000,765.0,206,365,0,1484
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,12,3,0.600000,0.056391,88.666667,NaN,NaN,88.666667,355.0,266,374,0,1616


In [60]:
# Percentage of Complex Words
import nltk
import os
from tqdm.notebook import tqdm

def total_word_counter(file_name):
    file_path = f"C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified\\{file_name}"
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        words = content.split()  # Split the content into words
        total_words = len(words)
    return total_words

def remove_stopwords_and_punctuation(file_name):
    file_path = f"C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified\\{file_name}"
    stop_words = set(stopwords.words('english'))
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

        # Tokenize the text into words
        words = nltk.word_tokenize(text)

        # Remove stop words and punctuation
        filtered_words = [word.lower() for word in words if (word.lower() not in stop_words) and (word.lower() not in string.punctuation)]

    return ' '.join(filtered_words)

excel_file_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Output Data Structure (1).xlsx"
output_df = pd.read_excel(excel_file_path)

folder_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified"
file_list = os.listdir(folder_path)

for index, file in tqdm(enumerate(file_list), total=len(file_list), desc="Completion"):
    # Calculate Average Sentence length for the current file
    remove_stopwords_and_punctuation(file)
    total_words = total_word_counter(file)
    complex_words = output_df.loc[index, 'COMPLEX WORD COUNT']
    try:
        output_df.loc[index, 'PERCENTAGE OF COMPLEX WORDS'] = complex_words/total_words
    except ZeroDivisionError:
        pass
output_df.to_excel(excel_file_path, index=False)
output_df

Completion:   0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\hp\AppData\Local\Temp\ipykernel_26908\2770803673.py:40: RuntimeWarning: divide by zero encountered in longlong_scalars
  output_df.loc[index, 'PERCENTAGE OF COMPLEX WORDS'] = complex_words/total_words


,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,26,5,0.677419,0.063265,490.000000,0.320408,NaN,490.000000,157,490,827,1,3133
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,52,31,0.253012,0.122239,679.000000,1.192931,NaN,679.000000,810,679,1375,0,5115
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,35,23,0.206897,0.094771,612.000000,1.406863,NaN,612.000000,861,612,1363,0,5048
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,35,70,-0.333333,0.178268,589.000000,1.436333,NaN,589.000000,846,589,1234,0,4713
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,18,8,0.384615,0.077612,335.000000,1.967164,NaN,335.000000,659,335,658,0,2628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,26,52,-0.333333,0.145522,536.000000,1.078358,NaN,536.000000,578,536,1001,2,3890
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,22,34,-0.214286,0.134293,417.000000,0.515588,NaN,417.000000,215,417,634,1,2675
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,4,2,0.333333,0.029126,206.000000,3.713592,NaN,206.000000,765,206,365,0,1484
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,12,3,0.600000,0.056391,88.666667,1.334586,NaN,88.666667,355,266,374,0,1616


In [61]:
# FOG Index
import nltk
import os
from tqdm.notebook import tqdm
    
excel_file_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Output Data Structure (1).xlsx"
output_df = pd.read_excel(excel_file_path)

folder_path = "C:\\Users\\hp\\OneDrive\\Desktop\\Analysis Files Modified"
file_list = os.listdir(folder_path)

for index, file in tqdm(enumerate(file_list), total=len(file_list), desc="Completion"):
    # Calculate subjectivity score for the current file
    avg_sen_len = output_df.loc[index, 'AVG SENTENCE LENGTH']
    per_complex_words = output_df.loc[index, 'PERCENTAGE OF COMPLEX WORDS']
    output_df.loc[index, 'FOG INDEX'] = 0.4*(avg_sen_len+per_complex_words)
output_df.to_excel(excel_file_path, index=False)
output_df

Completion:   0%|          | 0/100 [00:00<?, ?it/s]

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,26,5,0.677419,0.063265,490.000000,0.320408,196.128163,490.000000,157,490,827,1,3133
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,52,31,0.253012,0.122239,679.000000,1.192931,272.077172,679.000000,810,679,1375,0,5115
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,35,23,0.206897,0.094771,612.000000,1.406863,245.362745,612.000000,861,612,1363,0,5048
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,35,70,-0.333333,0.178268,589.000000,1.436333,236.174533,589.000000,846,589,1234,0,4713
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,18,8,0.384615,0.077612,335.000000,1.967164,134.786866,335.000000,659,335,658,0,2628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,26,52,-0.333333,0.145522,536.000000,1.078358,214.831343,536.000000,578,536,1001,2,3890
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,22,34,-0.214286,0.134293,417.000000,0.515588,167.006235,417.000000,215,417,634,1,2675
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,4,2,0.333333,0.029126,206.000000,3.713592,83.885437,206.000000,765,206,365,0,1484
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,12,3,0.600000,0.056391,88.666667,1.334586,36.000501,88.666667,355,266,374,0,1616
